---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def degree_dist(G):
    degrees = G.degree()
    degree_vals = sorted(set(degrees.values()))
    hist = [list(degrees.values()).count(i) / float(nx.number_of_nodes(G)) for i in degree_vals]
    
    return hist

degree_dist(P1_Graphs[0])

[0.465,
 0.237,
 0.1,
 0.056,
 0.032,
 0.026,
 0.016,
 0.014,
 0.009,
 0.007,
 0.005,
 0.005,
 0.002,
 0.001,
 0.001,
 0.002,
 0.002,
 0.003,
 0.002,
 0.001,
 0.002,
 0.002,
 0.002,
 0.001,
 0.002,
 0.001,
 0.002,
 0.001,
 0.001]

In [4]:
def graph_identification():
    algos = []
    
    for G in P1_Graphs:
        c = nx.average_clustering(G)
        sp = nx.average_shortest_path_length(G)
        dd = degree_dist(G)
        
        if len(dd) > 10:
            algos.append('PA')
        elif c < 0.1:
            algos.append('SW_H')
        else:
            algos.append('SW_L')  
        
    return algos

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [6]:
G.nodes(data=True)[:10]

[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (2, {'Department': 21, 'ManagementSalary': nan}),
 (3, {'Department': 21, 'ManagementSalary': 1.0}),
 (4, {'Department': 21, 'ManagementSalary': 1.0}),
 (5, {'Department': 25, 'ManagementSalary': nan}),
 (6, {'Department': 25, 'ManagementSalary': 1.0}),
 (7, {'Department': 14, 'ManagementSalary': 0.0}),
 (8, {'Department': 14, 'ManagementSalary': nan}),
 (9, {'Department': 14, 'ManagementSalary': 0.0})]

In [12]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

def salary_predictions():
    # Helper function for management / not management
    def is_management(node):
        managementSalary = node[1]['ManagementSalary']
        if managementSalary == 0:
            return 0
        elif managementSalary == 1:
            return 1
        else:
            return None

    df = pd.DataFrame(index=G.nodes())
    
    # Add all of the analyses
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['pr'] = pd.Series(nx.pagerank(G))
    
    df['is_management'] = pd.Series([is_management(node) for node in G.nodes(data=True)])
    
    # Train & test
    df_train = df[~pd.isnull(df['is_management'])]
    df_test = df[pd.isnull(df['is_management'])]
    
    features = ['clustering', 'degree', 'degree_centrality', 'closeness', 'betweeness', 'pr']
    
    X_train = df_train[features]
    Y_train = df_train['is_management']
    X_test = df_test[features]
    
    s = MinMaxScaler()
    X_train_scaled = s.fit_transform(X_train)
    X_test_scaled = s.fit_transform(X_test)
    
    clf = MLPClassifier()
    clf.fit(X_train_scaled, Y_train)
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]
    
    return pd.Series(test_proba, X_test.index)

salary_predictions()

1       0.432062
2       0.970181
5       0.999750
8       0.272799
14      0.885176
18      0.619093
27      0.751150
30      0.750509
31      0.413372
34      0.249335
37      0.251885
40      0.606810
45      0.468254
54      0.511689
55      0.519421
60      0.530947
62      0.999827
65      0.981824
77      0.131519
79      0.304012
97      0.175555
101     0.143685
103     0.502760
108     0.332325
113     0.926810
122     0.127197
141     0.938638
142     0.991694
144     0.090551
145     0.843096
          ...   
913     0.115480
914     0.139666
915     0.012856
918     0.228013
923     0.072339
926     0.121699
931     0.097470
934     0.029036
939     0.027532
944     0.012638
945     0.064526
947     0.165497
950     0.282182
951     0.076931
953     0.066354
959     0.012276
962     0.013631
963     0.320863
968     0.122146
969     0.106472
974     0.096092
984     0.027216
987     0.125342
989     0.121758
991     0.118950
992     0.014351
994     0.012209
996     0.0125

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [13]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [18]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

def new_connections_predictions():
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
        
    preferential_attachment = list(nx.preferential_attachment(G))
    
    df = pd.DataFrame(index=[(i[0], i[1]) for i in preferential_attachment])
    df['preferential_attachment'] = [i[2] for i in preferential_attachment]
    
    csh = list(nx.cn_soundarajan_hopcroft(G))
    
    df_csh = pd.DataFrame(index=[(i[0], i[1]) for i in csh])
    df_csh['cn_soundarajan_hopcroft'] = [i[2] for i in csh]
                          
    df = df.join(df_csh, how='outer')
                          
    df['cn_soundarajan_hopcroft'] = df['cn_soundarajan_hopcroft'].fillna(value=0)
    
    df['resource_allocation_index'] = [i[2] for i in list(nx.resource_allocation_index(G))]
    df['jaccard_coefficient'] = [i[2] for i in list(nx.jaccard_coefficient(G))]
                          
    df = future_connections.join(df, how='outer')
    
    # Train & test                    
    df_train = df[~pd.isnull(df['Future Connection'])]
    df_test = df[pd.isnull(df['Future Connection'])]
                          
    features = ['cn_soundarajan_hopcroft', 'preferential_attachment',
                'resource_allocation_index', 'jaccard_coefficient']
    
    X_train = df_train[features]
    Y_train = df_train['Future Connection']
    X_test = df_test[features]
    
    s = MinMaxScaler()
    X_train_scaled = s.fit_transform(X_train)
    X_test_scaled = s.fit_transform(X_test)
                          
    clf = MLPClassifier()
    clf.fit(X_train_scaled, Y_train)
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]

    predictions = pd.Series(test_proba, X_test.index)
    target = future_connections[pd.isnull(future_connections['Future Connection'])]
    target['prob'] = [predictions[i] for i in target.index]
    
    return target['prob']
                          
new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(107, 348)    0.041913
(542, 751)    0.012860
(20, 426)     0.951709
(50, 989)     0.012992
(942, 986)    0.013136
(324, 857)    0.012974
(13, 710)     0.395252
(19, 271)     0.305433
(319, 878)    0.013038
(659, 707)    0.012874
(49, 843)     0.013080
(208, 893)    0.012905
(377, 469)    0.008314
(405, 999)    0.023511
(129, 740)    0.020511
(292, 618)    0.025444
(239, 689)    0.013048
(359, 373)    0.010010
(53, 523)     0.178915
(276, 984)    0.013073
(202, 997)    0.013101
(604, 619)    0.101241
(270, 911)    0.013045
(261, 481)    0.144538
(200, 450)    0.999906
(213, 634)    0.012819
(644, 735)    0.077670
(346, 553)    0.012492
(521, 738)    0.011876
(422, 953)    0.021866
                ...   
(672, 848)    0.013045
(28, 127)     0.999012
(202, 661)    0.012437
(54, 195)     0.999999
(295, 864)    0.012953
(814, 936)    0.012839
(839, 874)    0.013136
(139, 843)    0.012965
(461, 544)    0.011456
(68, 487)     0.011385
(622, 932)    0.012921
(504, 936)    0.019494
(479, 528) 